# Perceptron

Here we explain the fundamental mechanism of discrete backpropagation on the following introductory machine learning model: the binarized perceptron or nonlinear regression. Before we begin, we must explain some fundamental concepts:

**Definition 1** - A matrix is said to be binarized if it only takes entries in $\{-1,+1\}$.

**Definition 2** - The Hamming Distance denoted $d$ of two binarized matrices $A$ and $B$ of equal size is given by: $$d(A,B)=\dfrac12\sum_{i} A_i + B_i$$

or in plain english "*the number of entries that are different between the two matrices*".

**Definition 3** - We typically denote a sample-wise ground truth $z$ and an input $x$ and batched ground truth $Z$ and an input $X$.

**Definition 4** - Given a binarized ground truth $Z$ and an input $X$, a binarized perceptron aims to find a binarized matrix $W$ and an integer vector $b$ to compute $\hat{Z}$ via:

$$\hat{Z} = \text{sign}(XW - b)$$

Where $\text{sign}$ is given by:

$$\text{sign}(x)=\begin{cases}-1\qquad \text{if }x<0\\ +1\qquad \text{if }x\geq 0 \end{cases}$$

Such that, $d(Z,\hat{Z})$ is minimized by correct choice of $W$ and $b$.

**Note 1** the matrix multiplication is taken to be ordinary matrix multiplication as one would do on integer matrices, if we map $-1\to 0$ and $+1\to 1$ in typical binary fashion - we replace the multiplication in the individual dot products with an bitwise XNOR and addition is then taken to be a population count/*Hamming weight* of the bit-string left after this XNOR'ing.

**Note 2** In practice, transistor-for-transistor *XOR* gates are simpler to implement and indeed in the experimental binary warp-level matrix multiply and accumulate instructions available on post-Turing Nvidia GPUs this is what actually gets used - and similarly in FPGA - however, the underlying mathematics of perceptron changes little and we can still train successfully in this regime.

This has problem has already been solved in various ways using either PlumerAi's Latent-Free approach or Bengio-et-al's Latent approach. In this notebook, I identify and explain a new approach which builds on PlumerAi's Latent-Free approach that I call "discrete optimization".

### Discrete Optimization

Discrete Optimization works on the following simple observation: that in the dot product of two binarized vectors, that the individual entries of both vectors can only work to either increment or decrement the final dot product.

This obvious fact, can be applied to find an optimization signal for the Perceptron. To simplify matters we'll first consider the case of a single sample $x$ and a single ground truth label $z$, for both we'll assume size 10. $\hat{z}$ is produced by the Perceptron formula and we can compare this against $z$.

```
z_hat = -1 +1 -1 -1 +1 -1 -1 -1 -1 -1 -1 -1
z     = -1 +1 -1 -1 -1 +1 -1 -1 -1 -1 -1 -1 
```

Here we have a few different kinds of behaviors which encode various signals:

1. Predicted: -1/+1, Truth: -1/+1 respectively, correct prediction - nothing to optimize here
2. Predicted: -1, Truth: +1, false negative - dot product between input $x$ and corresponding column in $W$ minus bias term $b$ was too **small**!
3. Predicted: +1, Truth: -1, false positive - dot product between input $x$ and corresponding column in $W$ minus bias term $b$ was too **big**!

We have a clear optimization signal here, we have to either increase or decrease the size of the value before we take its *sign*, we have two possible solutions:

1. The easiest is that we can simply increase/decrease the size of the bias to limit the effect of the dot product, in which ever way prevents the problem (too big vs too small)
2. Harder, is that we can look at the individual multiplications in the dot product and figure out which values in the $W$ column are contributing to this mess.

We call this process "blame attribution", where we figure out exactly which values we can blame for mispredictions. We cover the process of blame attribution for both the false positive and false negative case:

- False Positive:

Continuing our example, a false positive occurs at the $5^{\text{th}}$ entry of the $z/\hat{z}$ pair. This means that the dot product of $5^{\text{th}}$ column of $W$, denoted $W_5$ and $x$ exceeds that bias term $b_5$ - we can blame $b_5$ for being too small and attribute blame for it to increase in size. We can take a closer look at the values of $W_5$ and $x$ and attribute blame to the individual weights of this dot product:

```
W_5 = -1 +1 +1 -1 +1 +1 -1 +1 -1 +1
x   = -1 +1 -1 +1 -1 +1 -1 +1 -1 +1
```

$W_5$ can blamed for this unusual result in all of its entries except entries 3,4,5. So we attribute blame accordingly to these entries.

- False Negative:

In our example, the false negative occurs at the $6^{\text{th}}$ entry of the $z/\hat{z}$ pair. So we blame the bias term $b_6$ for being too large, and examine the dot product to $W_6$ as we did above to attribute blame:

```
W_6 = +1 -1 +1 +1 -1 +1 +1 -1 +1 -1 
x   = -1 +1 -1 +1 -1 +1 -1 +1 -1 +1
```

And now it is entries 4, 5, 6 that can be blamed for this false negative.

### Keeping Score

Now that we have our basic optimization signals, we can start tallying them, much in the same way as Plumerai's BOP tallies evidence determined from backpropagation. As a reminder, each weight is expected to have an unsigned blame counter that keeps track of the blame attributed to it as the network cycles through tasks, and similarly each bias term, however - this blame counter in this case is signed.

Once the blame counted, passes a certain threshold, either the weight is flipped or the bias is increment/decrement (dependent on sign). Most importantly, the process must also forgive blame by periodically incrementing/decrementing all counters towards 0 - this has the desired effect of removing erroneous blame that is attributed either as a result of noise in the data or from historical behaviour as the Perceptron is tuned.

With all of this, we are now ready to implement a binarized Perceptron.